## 1

In [ ]:
nominations = pd.read_csv("dados/nominations.csv")

ramon_rows = nominations[nominations["Nominee(s)"].str.contains("Ramón Menéndez Pidal")]
display(ramon_rows)

# store the ramon_rows in a csv file
ramon_rows.to_csv("ramon_rows.csv")

unique_nominators = set(ramon_rows["Nominator(s)"].unique())
#print(unique_nominators)

# split the nominators based on the \r\n character and create a new line for each split
split_entries = ramon_rows["Nominator(s)"].str.split("\r\n").explode().unique()
# remove the white spaces from the split_entries
split_entries = [entry.strip() for entry in split_entries]

print(split_entries)
# find the duplicates in the split_entries
print("=====================================")
# remove duplicates from the data list
print(set(data))

# data.append("Larissa")

print(f"Raw: {len(split_entries)} ==  My: {len(set(data))}")

def find_duplicates(input_list):
    seen = set()
    duplicates = set()
    for item in input_list:
        if item in seen:
            duplicates.add(item)
        else:
            seen.add(item)
    return list(duplicates)

# check witch entries are in the split_entries but not in the data
def difference(list1, list2):
    """
    This function returns the elements that are in list1 but not in list2.
    
    Args:
    list1 (list): The first list to compare.
    list2 (list): The second list to compare.

    Returns:
    list: A list containing elements that are in list1 but not in list2.
    """
    return [item for item in list1 if item not in list2]

print(find_duplicates(split_entries))
difference(set(data), split_entries)

In [ ]:
import pandas as pd
import networkx as nx

# Creating a DataFrame from the provided data
nominations = pd.read_csv("dados/nominations.csv")
nominations = nominations.copy()


data = []

def maisNomeado():
    G = nx.DiGraph()

    # Percorrendo cada linha do DataFrame
    for _, row in nominations.iterrows():
        nominators = [name.strip() for name in row["Nominator(s)"].replace('\r\n', '|').split('|')]
        nominees = [name.strip() for name in row["Nominee(s)"].replace('\r\n', ',').split(',')]
        year = row['Year']

        # Adicionando cada nomeado ao grafo
        for nominee in nominees:
            for nominator in nominators:
                if G.has_edge(nominator, nominee):
                    G[nominator][nominee]['years'].add(year)
                else:
                    G.add_edge(nominator, nominee, years={year})

    # Contagem dos nominadores únicos por nomeado
    nominee_counts = {}
    
    for nominator, nominee in G.edges():
        if nominee not in nominee_counts:
            nominee_counts[nominee] = set()
        nominee_counts[nominee].add(nominator)

    # Encontrar o nomeado com o maior número de nominadores únicos
    most_nominated = None
    max_nominators = 0
    for nominee, nominators in nominee_counts.items():
        if len(nominators) > max_nominators:
            most_nominated = nominee
            max_nominators = len(nominators)

    return most_nominated, max_nominators

maisNomeado()

## 2


In [ ]:
import pandas as pd
import networkx as nx
from collections import defaultdict

# Carrega os dados novamente
nominations = pd.read_csv("dados/nominations.csv")

def nomeacoesCruzadas():
    # Criar o grafo direcionado
    G = nx.DiGraph()

    # Adiciona as arestas baseadas nas nomeações
    for _, row in nominations.iterrows():
        nominators = [name.strip() for name in row["Nominator(s)"].replace('\r\n', '|').split('|')]
        nominees = [name.strip() for name in row["Nominee(s)"].replace('\r\n', ',').split(',')]
        category = row["Category"]
        for nominator in nominators:
            for nominee in nominees:
                # Adiciona uma aresta do nominador para o nomeado
                G.add_edge(nominator, nominee, category=category)
    
    # Encontrar o maior componente fortemente conectado
    largest_scc = max(nx.strongly_connected_components(G), key=len)
    
    # Coletar categorias envolvidas neste componente
    categories = set()
    for u, v, data in G.edges(data=True):
        if u in largest_scc and v in largest_scc:
            categories.add(data['category'])

    # Retorna o tamanho do maior SCC e as categorias envolvidas
    return len(largest_scc), categories

# Chamada da função
print(nomeacoesCruzadas())

## 3

In [ ]:
import pandas as pd
import networkx as nx

# Carregando os dados
nominations = pd.read_csv("dados/nominations.csv")

def caminhoEinsteinFeynman() -> list[str]:

    local_nominations = nominations.copy()
    
    # Filtrar dados para o período entre 1921 e 1965 e para a categoria de Física
    local_nominations = local_nominations[(local_nominations['Year'] >= 1921) & (local_nominations['Year'] <= 1965) & (local_nominations['Category'] == 'Physics')]
    
    # Criando o grafo
    G = nx.DiGraph()
    
    for idx, row in local_nominations.iterrows():
        nominators = [name.strip() for name in row["Nominator(s)"].replace('\r\n', '|').split('|')]
        nominees = [name.strip() for name in row["Nominee(s)"].replace('\r\n', ',').split(',')]
        
        # Adicionando arestas de cada nominador para cada nomeado
        for nominator in nominators:
            for nominee in nominees:
                G.add_edge(nominator, nominee)
    
    nx.write_graphml(G, "Albert2Feyman.graphml")

    # Encontrando o caminho entre Einstein e Feynman
    # Einstein como nominador (teremos que verificar se ele nomeou alguém diretamente)
    # Feynman como nomeado
    try:
        # Inicialmente assumimos que Einstein e Feynman são parte dos nodos
        # Este método levanta uma exceção se não houver caminho
        # Source and target nodes
        source = 'Albert Einstein'
        target = 'Richard Phillips Feynman'

        # Find all shortest paths
        all_shortest_paths = list(nx.all_shortest_paths(G, source=source, target=target))
        # Extract only intermediate nodes from each path
        all_shortest_paths = [path[1:-1] for path in all_shortest_paths]  # Slicing to exclude the first and last elements
        return all_shortest_paths
    except nx.NetworkXNoPath:
        return []  # Retornar lista vazia se não houver caminho


# Teste da função
caminhoEinsteinFeynman()